<a href="https://colab.research.google.com/github/alanbseo/LCAworkshop/blob/master/LCA_1_2_CoalLCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install the BW package
!pip install brightway25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 30.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.0/305.0 kB 26.1 MB/s eta 0:00:00
  Created wheel for brightway25: filename=brightway25-1.0.6-py3-none-any.whl size=3453 sha256=d282e63c0b4f7fe8d14eb6090c91b47174968702c54d3

In [ ]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as bwa

import os

In [ ]:
# Exercise: CO2 emissions from 1 kWh coal-fired electricity in the US

# In our technosphere matrix, we will have four production exchanges (each of amount 1),
# and three consumption exchanges.
# Our biosphere matrix will only have one number, the emission of Iron ore from Iron mining.


# activity node
coal_mining = 100
rail_transportation = 201
electricity_generation = 101
electricity = 102
petroleum_refining = 103









# Technosphere
sb_t_data = np.array([
    1,   # production of iron mining
    1,   # production of coal
    1,   # production of steel
    1,   # production of steel bike
    1.1, # input of iron mining
    0.2, # input of coal
    8,   # input of steel
])

# We need integer indices for each concept. The numbers themselves are arbitrary,
# the point is that we will use them consistently in the datapackages, and maintain a mapping from our concepts to these numbers.

sb_t_indices = np.array([
    (101, 101), # production of iron mining
    (100, 100), # production of coal
    (102, 102), # production of steel
    (103, 103), # production of steel bike
    (101, 102), # input of iron mining
    (100, 102), # input of coal
    (102, 103), # input of steel
    ],
    dtype=bwp.INDICES_DTYPE
)
sb_t_flip = np.array([False, False, False, False, True, True, True]) # Numerical sign of the inputs needs to be flipped negative

# Biosphere
sb_b_data = np.array([1.25])
sb_b_indices = np.array([
    (201, 101), # emission of Iron ore
    ],
    dtype=bwp.INDICES_DTYPE
)

# Characterization matrix

sb_c_data = np.array([1])
sb_c_indices = np.array([
    (201, 201), # Iron ore
    ],
    dtype=bwp.INDICES_DTYPE
)



In [ ]:
# The main interface for using this library is the Datapackage class. You should use the utility functions create_datapackage and load_datapackage.
# A datapackage is a set of file objects (either in-memory or on disk) that includes a metadata file object, and one or more data resource files objects.
# The metadata file object includes both generic metadata (i.e. when it was created, the data license) and metadata specific to each data resource (how it can be used in calculations, its relationship to other data resources).

sb_dp_static = bwp.create_datapackage()

sb_dp_static.add_persistent_vector(
    matrix='technosphere_matrix',
    indices_array=sb_t_indices,
    data_array=sb_t_data,
    flip_array=sb_t_flip,
)
sb_dp_static.add_persistent_vector(
    matrix='biosphere_matrix',
    indices_array=sb_b_indices,
    data_array=sb_b_data,
)
sb_dp_static.add_persistent_vector(
    matrix='characterization_matrix',
    indices_array=sb_c_indices,
    data_array=sb_c_data,
)

# Base class for single and multi LCA classes
sb_lca = bc.LCA(
    demand={steelbike: 1},
    data_objs=[sb_dp_static],
)

# The LC inventory. Calculate a life cycle inventory.
sb_lca.lci()
# 1) Load LCI data, and construct the technosphere and biosphere matrices.
# 2) Build the demand array
# 3) Solve the linear system to get the supply array and life cycle inventory.

sb_lca.lcia() # Calculate the life cycle impact assessment.
# 1) Load and construct the characterization matrix
# 2) Multiply the characterization matrix by the life cycle inventory

sb_lca.score # The LCIA score as a float.
# means 1.25 iron ore * 1.1 kg iron mining * 8 kg steal